In [1]:
from support import CalculateLabelsCorrelationWithFTest
from skmultilearn.dataset import load_dataset

In [10]:
train_data = load_dataset("scene", "train")
test_data = load_dataset("scene", "test")
# let's use the same "scene" dataset, that was used in the previous notebook (`1_first_model_experiments.ipynb`)

train_data = load_dataset("scene", "train")
_, y_train, _, _ = train_data

scene:train - exists, not redownloading
scene:test - exists, not redownloading
scene:train - exists, not redownloading


In [13]:
ccwf = CalculateLabelsCorrelationWithFTest(alpha=1)
res = ccwf.get(y_train)


In [14]:
res

,for_label,expand_this_label,f_test_result
0,0,2,56.728266
1,0,3,56.368680
2,0,1,45.657072
3,0,5,33.173567
4,0,4,30.068018
5,1,4,59.336173
6,1,0,45.657072
7,1,5,44.889245
8,1,2,38.222988
9,1,3,37.984223


In [22]:
def build_chain_based_on_f_test(res):
    chain = []
    sorted_res = res.sort_values(by=["f_test_result"], ascending=False)
    
    element = int(sorted_res.iloc[0]["for_label"])
    chain.append(element)
    
    j = 0

    while j < 10:
        print(f"... {j}")
        j += 1

        m = ~sorted_res["expand_this_label"].isin(chain)
        m &= sorted_res["for_label"] == element

        if m.sum() == 0:
            break

        sliced_res = sorted_res[m]
        sorted_sliced_res = sliced_res.sort_values(by=["f_test_result"], ascending=False)

        element = int(sorted_sliced_res.iloc[0]["expand_this_label"])
        chain.append(element)
    
    return chain

build_chain_based_on_f_test(res)


... 0
... 1
... 2
... 3
... 4
... 5


[5, 4, 1, 0, 2, 3]